# Wrangle Zillow

### If you are finished with the exercises...here is what I want y'all to do next...this will help you prepare for the regression project :
- use mySQL to query the zillow database.
- you will need to use the properties_2017 and predictions_2017 tables.
- you will want to gather the following information (and figure out which columns are most likely to give you this information)...square feet of the house ("living square feet"), number of bedrooms, number of bathrooms, the assess value of the house by the tax appraisal district ('taxvaluedollarcnt'...this will be your target variable in the project), and 2-3 other variables you think will be useful (think about what you know about what drives home prices, and also the amount of non-null values available in selecting your extra variables.
- you will want to limit your data to properties that were sold in may, june, july or august (date is in predictions_2017 table).
- There are a ton of rows...so think about how you can limit your data set early to keep your query going! i.e. filter the dates!

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd 
import numpy as np 
import os
from env import host, user, password 
import wrangle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
#Connection function to access Codeup Database and retrieve zillow dataset from mysql
def get_connection(db, user=user, host=host, password=password):
    '''
    This function creates a connection to Codeup Database with 
    info from personal env file (env file has user login information).
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'  

##############ACQUIRE##################

def acquire_zillow():
    '''
    This function reads in the zillow data from the Codeup 
    Database connection made from get_connection
    and returns a pandas DataFrame with all columns.
    '''
    sql_query = '''
                SELECT parcelid, bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet,
                taxvaluedollarcnt
                FROM  properties_2017
                JOIN predictions_2017 USING(parcelid)
                WHERE transactiondate between "2017-05-01" and "2017-08-31"
                and unitcnt = 1;
                '''
    
    return pd.read_sql(sql_query, get_connection('zillow'))    



def get_zillow_data(cached=False):
    '''
    This function reads in zillow data from Codeup database and 
    writes data to a csv file if cached == False. If cached == True 
    reads in zillow df from a csv file, returns df.
    '''
    if cached == False or os.path.isfile('zillow.csv') == False:
        
        # Read fresh data from db into a DataFrame.
        df = acquire_zillow()
        
        # Write DataFrame to a csv file.
        df.to_csv('zillow.csv')
        
    else:
        
        # If csv file exists or cached == True, read in data from csv.
        df = pd.read_csv('zillow.csv', index_col=0)
        
    return df

In [3]:
#functions to get specific zillow data
def acquire_some_more_zillow():
    '''
    This function reads in the zillow data from the Codeup 
    Database connection made from get_connection
    and returns a pandas DataFrame with all columns.
    '''
    sql_query = '''
                SELECT parcelid, bedroomcnt, bathroomcnt, buildingqualitytypeid, yearbuilt, 
                regionidcounty, fips, calculatedfinishedsquarefeet, taxamount, taxvaluedollarcnt
                FROM  properties_2017
                JOIN predictions_2017 USING(parcelid)
                WHERE transactiondate between "2017-05-01" and "2017-08-31"
                and unitcnt = 1;
                '''
    return pd.read_sql(sql_query, get_connection('zillow')) 


def get_some_more_zillow_data(cached=False):
    '''
    This function reads in zillow data from Codeup database and 
    writes data to a csv file if cached == False. If cached == True 
    reads in zillow df from a csv file, returns df.
    '''
    if cached == False or os.path.isfile('zillow.csv') == False:
        
        # Read fresh data from db into a DataFrame.
        df = acquire_zillow_data()
        
        # Write DataFrame to a csv file.
        df.to_csv('zillow.csv')
        
    else:
        
        # If csv file exists or cached == True, read in data from csv.
        df = pd.read_csv('zillow.csv', index_col=0)
        
    return df

In [4]:
#features for some zillow data: "parcelid", "bedroomcnt", "bathroomcnt", "buildingqualitytypeid", "yearbuilt", "regionidcounty","fips","calculatedfinishedsquarefeet", "taxamount", "taxvaluedollarcnt"
#sql query:
#SELECT parcelid, bedroomcnt, bathroomcnt, buildingqualitytypeid, yearbuilt, regionidcounty, fips, calculatedfinishedsquarefeet, taxamount, taxvaluedollarcnt
#FROM properties_2017
#JOIN predictions_2017 using(parcelid)
3#WHERE transactiondate between "2017-05-01" and "2017-06-30"
#and unitcnt = 1;

3

## Summarize
- head()
- describe()
- info()
- isnull()
- value_counts()
- shape

In [5]:
df = get_zillow_data(cached=False)

In [6]:
df.head()

,parcelid,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt
0,11721753,3.0,2.0,1316.0,205123.0
1,11289917,3.0,2.0,1458.0,136104.0
2,11637029,3.0,2.0,1766.0,810694.0
3,11705026,2.0,1.0,1421.0,35606.0
4,11446756,3.0,2.0,1491.0,107110.0


In [7]:
df.describe()

,parcelid,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt
count,2.495000e+04,24950.00000,24950.000000,24948.000000,2.495000e+04
mean,1.190844e+07,2.96016,2.251864,1709.276816,4.807596e+05
std,3.349340e+06,1.00650,0.993814,944.327658,6.830046e+05
min,1.071186e+07,0.00000,0.000000,240.000000,1.000000e+03
25%,1.117189e+07,2.00000,2.000000,1144.000000,1.880000e+05
50%,1.179469e+07,3.00000,2.000000,1480.000000,3.349620e+05
75%,1.249927e+07,4.00000,3.000000,1988.000000,5.439060e+05
max,1.676560e+08,11.00000,11.000000,35640.000000,2.385837e+07


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24950 entries, 0 to 24949
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      24950 non-null  int64  
 1   bedroomcnt                    24950 non-null  float64
 2   bathroomcnt                   24950 non-null  float64
 3   calculatedfinishedsquarefeet  24948 non-null  float64
 4   taxvaluedollarcnt             24950 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 974.7 KB


### .info() takeaways
- bedroom and bathroom count are floats, look into if there are really half bedrooms.  I know half bathrooms exist, but I should still check it out later in value counts.  

In [10]:
df.isnull().sum()

parcelid                        0
bedroomcnt                      0
bathroomcnt                     0
calculatedfinishedsquarefeet    2
taxvaluedollarcnt               0
dtype: int64

### isnull().sum() takeaways
- only two nulls in square feet column, can probably drop those rows

In [11]:
df.isna().sum()

parcelid                        0
bedroomcnt                      0
bathroomcnt                     0
calculatedfinishedsquarefeet    2
taxvaluedollarcnt               0
dtype: int64

### isna().sum() takeaways
- looks like the same two from isnull but be sure they are differnt values after dropping

In [14]:
def get_value_counts(df):
    for column in df:
        print(df.column.value_counts())

In [15]:
get_value_counts(df)

AttributeError: 'DataFrame' object has no attribute 'column'

## Zillow Planning:

## Phase 1


### First Iteration Features
- square feet
- number of bedrooms
- number of bathrooms
- taxvaluedollarcnt (target)

## Phase 2

### Change to int:
- bedroomcnt
- yearbuilt
- regionidcounty
- fips


### Encode
- fips
- regionidcounty

### New features to make:
- residence_age = how old the residence is
- 

In [ ]:
df.regionidcounty.value_counts()

In [ ]:
df.fips.value_counts()

In [ ]:
df.bathroomcnt.value_counts()

In [ ]:
df.describe()

In [ ]:
df.columns